<a href="https://colab.research.google.com/github/ANMOL2001A/Mario_game_bot/blob/main/Mario_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch
from torch.utils.data import Dataset, DataLoader
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
import tqdm
import torch

class ChatData(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r"))
        self.dialogues = []

        for conv in self.data:
            dialogue = ""
            for turn in conv["dialog"]:
                if "content" in turn:
                    dialogue += f"{turn['role']}: {turn['content']} "
            self.dialogues.append(dialogue.strip())

        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        return self.dialogues[idx]

def infer(inp, model, tokenizer, device):
    inp = f"<startofstring> {inp} <bot>:"
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output

def train(chatData, model, optim, device):
    epochs = 12

    for epoch in range(epochs):
        data_loader = DataLoader(chatData, batch_size=64, shuffle=True)
        for batch in tqdm.tqdm(data_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            X = chatData.tokenizer(batch, max_length=40, truncation=True, padding="max_length", return_tensors="pt")
            X_ids = X["input_ids"].to(device)
            X_mask = X["attention_mask"].to(device)
            optim.zero_grad()
            loss = model(X_ids, attention_mask=X_mask, labels=X_ids).loss
            loss.backward()
            optim.step()

        torch.save(model.state_dict(), "model_state.pt")

        # Print example inference at the end of each epoch
        example_input = "Tell me about Mario."
        print(infer(example_input, model, chatData.tokenizer, device))

# Set up tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", "bos_token": "<startofstring>", "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Set up DataLoader and optimizer
chatData = ChatData("/content/mario_conversational_dataset.json", tokenizer)
optim = Adam(model.parameters(), lr=1e-3)

# Train the model
train(chatData, model, optim, device)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 121.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00


(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

 Tell me about Mario.  <bot>:  tips tips tips tips tips tips tips tips tips tips tips tips


Epoch 11/12: 100%|██████████| 1/1 [00:00<00:00, 13.83it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Tell me about Mario.  <bot>:  game game game game game game game game game game game game


Epoch 12/12: 100%|██████████| 1/1 [00:00<00:00, 13.02it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Tell me about Mario.  <bot>:  game game game game game game game game game game game game


In [ ]:
print("Infer from model:")
while True:
    inp = input()
    print(infer(inp, model, tokenizer, device))

Infer from model:
"Tell me more about Bowser."


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 "Tell me more about Bowser."  <bot>:  game game game game game game game game game game
